# local v: di

Lets recap the last 7 weeks in the spirit of a [coda](https://en.wikipedia.org/wiki/Coda_(music)):

* `help`
   * the help command followed by any command you wish to further explore 
   * e.g., `h twoway`

* `tokenize`
   * represent a numerical sequence (e.g. 1,2,3,...,6)  
   * as a sequence of letters (e.g. a,b,c,...,z or A,B,C,...,Z)

* `quietly`
   * how to control what is displayed in the Stata terminal
   * Stata uses this e.g. with the neat regression output, and with profile.do
   * may very easily be changed to `noisily` if that suits your needs

* `delimit`
   * neat way to mimic `SAS` in defining the end of **one "line" of code**
   * especially useful when dealing with graphical output and numerous graphing options
   * absolutely necessary if ever you wish to translate `.SAS` code into `.do`
   * may make accessible what otherwise were inaccessible datasets (e.g. [NHANES](https://pubmed.ncbi.nlm.nih.gov/?term=nhanes))

* `rnormal()`
   * simulating parametric distributions
   * you may develop this idea on your own in the future
   * `runiform()` & `rnormal()` are the only examples demonstrated in this class

* `pwd`
   * pwd
   * di c(pwd)
   * di "text with embedded `c(pwd)' for whatever reason"
   * global filepath c(pwd)

* `r(mean)`
   * creturn list
   * return list
   * ereturn list

* `by`
   * collapse (`statistic`) varname1 by(varname2)
   * egen varname1=`statistic`(varname2)
   * above commands are **equivalent** in many regards
   * one distorts the data; the other doesn't do so at all
   * useful created twoway plots or other graphical output

* `program define`
   * rigid, specific programs (without `syntax varlist` or `syntax, options`)
   * flexible, generalizable programs (with `syntax [varlist]`, `[options]`)

* `capture`
   * preventing error results and an arrested script
   * Stata is finicky, *pernickety* and this little preface to a command may spare you untold misery
   * we've used it most frequently before `program define` & `log using`
   * but, notably, we've used it as a parsimonous variant in `syntax varlist if`

* `twoway`
   * last weeks class, which we didn't cover because of `hw1`-related issues
   * notes have been updated to give you the best bang for your buck 
   * lookout for the following **very** important issues:
     * workflow;
     * open science; and,
     * chatGPTs take on these.

* on your own, please go walk step-by-step, command-by-command, through the `twoway` examples offered

* think about `if macro {` conditional code-blocks and your deployment of this artifice in future, very powerful Stata scripts!

Today we'll circle back to the method emplyed throughout this class:

   * defining macros
   * formatting them
   * [embedding](https://jhustata.github.io/book/ggg.html) them in strings, text, figures, and excel
   * producing aesthically pleasing, richly informative output

But to what end?

   * the advanced Stata Programming class answers that question thus:
       * open science
       * self-publication
       * collaboration
       * etc... 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
#import numpy as np
#import sklearn as skl
# 

#plt.figure(figsize=[2, 2])
G = nx.DiGraph()
G.add_node("workflow",  pos = (0,700) )
G.add_node("jupyter",  pos = (-2000, 960) )
G.add_node("dofile",  pos = (2000, 950) )
G.add_node("python ", pos = (-3000, 550) )
G.add_node("stata", pos = (3000, 550) )
G.add_node("build", pos = (-1900, 150) )
G.add_node("dyndoc", pos = (1900, 150) )
G.add_node("html", pos = (0,0))
G.add_node("push", pos = (0, -475))
G.add_node("ghp", pos = (0, -950))
G.add_edges_from([ ("jupyter","python "), ("dofile", "stata")])
G.add_edges_from([("python ", "build"), ("stata", "dyndoc") ])
G.add_edges_from([("build", "html"), ("dyndoc", "html")])
G.add_edges_from([("html","push")])
G.add_edges_from([("push","ghp")])
nx.draw(G, 
        nx.get_node_attributes(G, 'pos'), 
        with_labels=True, 
        font_weight='bold', 
        node_size = 4500,
        node_color = "lightblue",
        linewidths = 3)
ax= plt.gca()
ax.collections[0].set_edgecolor("#000000")
ax.set_xlim([-5000, 5000])
ax.set_ylim([-1000, 1000])
plt.show()

If at all you've enjoyed the conveniences brought forth by this classbook, then it would be remiss of me not to let you know from whence this book comes:

* a data science [class](https://www.jhsph.edu/courses/course/36630/2022/140.628.41/data-science-for-public-health-i)
* python environment
* jupyter notebooks
* vscode by miscrosoft
* dofiles
* stata

In the [advanced](https://jhustata.github.io/class700/intro.html) [class](https://www.jhsph.edu/courses/course/37447/2022/340.700.71/advanced-stata-programming) we merge these ideas and publish our analyses and documentation (roll over[,](https://www.youtube.com/watch?v=kKlzUKLOm_U) annotation) online via [gh-pages](https://pages.github.com)!!!

A word on **simulation**: 

* worth developing skills
* making up data
* imaginary scenarios
* and why would one do that?
    * missing data
    * bootstrap
    * cryptography/disclosure risk
	* sample-size calculation (variant on missing data)
	* etc.

```stata
qui {
	clear 
	cls
	if c(N) { //background
		inspired by polack et al. nejm 2020
		NEJM2020;383:2603-15
		lets do some reverse engineering
		aka simulate, generate data 
		from results: reversed process!!
	}
	if c(N)<1 { //methods
		if c(os)=="Windows" {
			global workdir "`c(pwd)'\"
		}
		else {
			global workdir "`c(pwd)'/"
		}
		capture log close
		log using ${workdir}simulation.log, replace 
		set seed 340600
		set obs 37706
	}
	if c(N)==37706 { //simulation 
	    #delimit ; 
		//row1
		g bnt=rbinomial(1,.5);
		lab define Bnt 
		    0 "Placebo"  
	        1 "BNT162b2" ;
		label values bnt Bnt ;
		tab bnt ; 
		//row2
		gen female=rbinomial(1, .494); 
		label define Female  
		    0 "Male"  
			1 "Female"; 
		label values female Female; 
		tab female;
		//row3 
		tempvar dem ;
		gen `dem'=round(runiform(0,100),.1); 
		recode `dem'  
		    (0/82.9=0)  
		    (83.0/92.1=1)  
		    (92.2/96.51=2)   
		    (96.52/97.0=3)  
		    (97.1/97.2=4)  
		    (97.3/99.41=5)  
		    (99.42/100=6)  
		         , gen(race);
		lab define Race  
			0 "White"    
		    1 "Black or African American" 
			2 "Asian" 
			3 "Native American or Alsak Native"  
			4 "Native Hawaiian or other Pacific Islander"  
			5 "Multiracial"  
			6 "Not reported"; 
		label values race Race; 
		tab race;
		//row4 
		gen ethnicity=rbinomial(1,0.28);
		tostring ethnicity, replace;
		replace ethnicity="Latinx" if ethnicity=="1";
		replace ethnicity="Other" if ethnicity=="0";
		//row5 
		tempvar country;
		gen `country'=round(runiform(0,100), .1);
		recode `country'   
		    (0/15.3=0)  
			(15.4/21.5=1)  
			(21.6/23.6=2)  
			(23.7/100=3) 
			    , gen(country) ;
		label define Country 
			0 "Argentina"  
		    1 "Brazil"  
			2 "South Africa"  
			3 "United States"; 
		label values country Country; 
		tab country;
		//row7 
		gen age=(rt(_N)*9.25)+52 ; 
		replace age=runiform(16,91)  
		    if !inrange(age,16,91); 
		summ age, d ;
		local age_med=r(p50); local age_lb=r(min); local age_ub=r(max);
		gen dob = d(27jul2020) -  
		          (age*365.25) ; 
		gen dor = dob + age*365.25 + runiform(0,4*30.25); 
		//row6 
		gen over55=age>55 ; tab over55;
		//row8 
		gen bmi=rbinomial(1, .351); tab bmi; 
		//figure 3 
		g days=rweibull(.7,17,0) if bnt==0 ;
		g covid=rbinomial(1, 162/21728) if bnt==0 ; 
		replace days=rweibull(.4,.8,0) if bnt==1 ;
		replace covid=rbinomial(1, 14/21772) if bnt==1; 
		//key dates 
		gen eft = dor + days;
		//date formats
		format dob %td; format dor %td; format eft %td;
		 //kaplan-meier curve
		 stset days, fail(covid) ;
		 sts graph,  
		    by(bnt)  
		    fail per(100)  
		    tmax(119)  
		    xlab(0(7)119) 
		    ylab(0(.4)2.4, 
		        angle(360)    
			    format("%3.1f")
				)  
		    xti("Days after Dose 1")  
		    legend(off) 
		    text(
			    2.3 100 
			    "Placebo",  
			     col(navy)
				 )  
		    text(
			    .5 100 
				"BNT162b2",  
			    col(maroon)
				) ;
		graph export BNT162b2.png, replace ;
		stcox bnt ;
		drop _* age over55 days ;
		g bnt_id=round(runiform(37,37+_N)) ;
		compress  ;
		#delimit cr
		//label variables
		lab var bnt_id "Participant Identifier"
		lab var bnt "Random treatment assignment"
		lab var female "Gender at birth"
		lab var race "Self-identified race"
		lab var ethnicity "Hispanic ethnicity"
		lab var country "Country where trial was conducted"
		lab var dob "Date of birth"
		lab var dor "Date of recruitment into BNT162b2 trial"
		lab var eft "Date of exit from BNT162b2 trial"
		lab var bmi "Obese"
		lab var covid  "Covid-19 status on eft date"
		//label data
		lab data "Safety and Efficacy of the BNT162b2 mRNA Covid-19 Vaccine"
		describe
		order bnt_id dob female race ethnicity country bmi bnt eft covid 
		*replace eft=. if eft>d(15dec2020) //some folks lost to followup
		save BNT162b2, replace 

	}
  log close 
}		
```